In [1]:
#installing kaggle library
! pip install kaggle


In [2]:
# configuring path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
#api command
!kaggle datasets download -d bwandowando/3-million-instagram-google-store-reviews


Dataset URL: https://www.kaggle.com/datasets/bwandowando/3-million-instagram-google-store-reviews
License(s): CC0-1.0
3-million-instagram-google-store-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
#extracting zip file
from zipfile import ZipFile
dataset = '/content/3-million-instagram-google-store-reviews.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('extracted')

extracted


In [5]:
#required libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [8]:
df=pd.read_csv('/content/INSTAGRAM_REVIEWS.csv',encoding= 'ISO-8859-1')

In [9]:
df.shape

(3079939, 9)

In [10]:
df.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,Ù Ø³*******ÙÙ,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,NaN,2022-01-05 23:39:32
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,NaN,2022-01-05 23:39:50
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08


In [11]:
df.isnull().sum()

,0
Unnamed: 0,0
review_id,0
pseudo_author_id,0
author_name,0
review_text,128
review_rating,0
review_likes,0
author_app_version,765552
review_timestamp,0


In [12]:
df.dropna(subset=['review_text'], inplace=True)  # Drop missing reviews

df['author_app_version'] = df['author_app_version'].fillna('unknown') # Fill Nan Values

In [13]:
df.isnull().sum()


,0
Unnamed: 0,0
review_id,0
pseudo_author_id,0
author_name,0
review_text,0
review_rating,0
review_likes,0
author_app_version,0
review_timestamp,0


In [14]:
df.shape

(3079811, 9)

In [15]:
df.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,Ù Ø³*******ÙÙ,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,unknown,2022-01-05 23:39:32
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,unknown,2022-01-05 23:39:50
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08


In [16]:
df['review_rating'].value_counts()

,count
review_rating,
5,1963214
1,589844
4,246602
3,155200
2,124951


In [17]:
#labelling the ratings
def label_sentiment(rating):
    if rating >= 4:
        return 'positive review'
    elif rating == 3:
        return 'neutral review'
    else:
        return 'negative review'

In [18]:
df['sentiment'] = df['review_rating'].apply(label_sentiment)
print(df['sentiment'].value_counts())

sentiment
positive review    2209816
negative review     714795
neutral review      155200
Name: count, dtype: int64


In [19]:
port_stem=PorterStemmer()
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [20]:
df['stemmed_content']=df['review_text'].apply(stemming)

In [21]:
# Clean missing stemmed_content first
df = df[df['stemmed_content'].notna()]
df = df[df['stemmed_content'].str.strip() != '']


In [22]:
df.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp,sentiment,stemmed_content
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15,negative review,use app year great way escap overli polit envi...
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,Ù Ø³*******ÙÙ,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29,positive review,khayemalo sag bgad
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,unknown,2022-01-05 23:39:32,positive review,love
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,unknown,2022-01-05 23:39:50,positive review,best
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08,positive review,super


In [23]:
from sklearn.utils import resample

df_pos = df[df['sentiment'] == 'positive review']
df_neg =df[df['sentiment'] == 'negative review']
df_neu = df[df['sentiment'] == 'neutral review']

df_neg_upsampled = resample(df_neg,
                            replace=True,
                            n_samples=len(df_pos),
                            random_state=42)

df_neu_upsampled = resample(df_neu,
                            replace=True,
                            n_samples=len(df_pos),
                            random_state=42)

df_balanced = pd.concat([df_pos, df_neg_upsampled, df_neu_upsampled])
print(df_balanced['sentiment'].value_counts())


sentiment
positive review    2054215
negative review    2054215
neutral review     2054215
Name: count, dtype: int64


In [24]:
#converting textual data to numerical data
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=20000)

X = vectorizer.fit_transform(df_balanced['stemmed_content'])  # ✅ Don't overwrite this!





In [25]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df_balanced['sentiment'])


In [26]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 43976487 stored elements and shape (6162645, 20000)>
  Coords	Values
  (1, 10149)	1.0
  (2, 2372)	1.0
  (3, 16893)	1.0
  (4, 12591)	1.0
  (5, 3937)	0.2601588142826362
  (5, 12396)	0.3097400807022021
  (5, 19164)	0.16414650840720105
  (5, 2707)	0.2210247970955239
  (5, 8119)	0.10294496957749756
  (5, 15609)	0.18165493227662585
  (5, 3688)	0.23157613788418796
  (5, 9623)	0.12785408305653606
  (5, 4151)	0.17208543987456143
  (5, 12722)	0.17069429351753282
  (5, 14998)	0.33399079907144336
  (5, 16031)	0.28407889522423485
  (5, 5165)	0.2787502283317647
  (5, 8432)	0.3260243646260865
  (5, 9656)	0.3274773009205221
  (5, 4187)	0.32369199782418684
  (6, 8513)	1.0
  (7, 2372)	0.2925117786608254
  (7, 8119)	0.21322679056190114
  (7, 11870)	0.3233939469894127
  (7, 1179)	0.16525342671446308
  :	:
  (6162643, 10104)	0.17337562654994984
  (6162643, 18486)	0.24271740280382934
  (6162643, 8797)	0.32979899602091767
  (6162643, 19008)	0.2612

In [27]:
print(y)

[2 2 2 ... 1 1 1]


In [48]:


X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, df_balanced.index,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [49]:
print(X.shape,X_train.shape,X_test.shape)

(6162645, 20000) (4930116, 20000) (1232529, 20000)


In [50]:
print(X_train.shape[0], y_train.shape[0])


4930116 4930116


In [51]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 35189436 stored elements and shape (4930116, 20000)>
  Coords	Values
  (1, 2745)	0.4403517334114406
  (1, 6543)	0.4979298974577644
  (1, 2786)	0.7470984995964318
  (2, 3839)	0.19268565577711258
  (2, 5732)	0.13190871435505372
  (2, 12950)	0.11982067175382058
  (2, 200)	0.24650638726262628
  (2, 11363)	0.29069115126506306
  (2, 13517)	0.12851830060229633
  (2, 12982)	0.16264111108748808
  (2, 5784)	0.19621772749378835
  (2, 13190)	0.18767729686118997
  (2, 3390)	0.1923241335212195
  (2, 14104)	0.18023269660508648
  (2, 15946)	0.2740370272399657
  (2, 11365)	0.4459928815031376
  (2, 3848)	0.245623579829194
  (2, 14123)	0.30282848540811264
  (2, 237)	0.32667595342484773
  (2, 368)	0.25869632665727554
  (3, 1179)	0.09645435233422335
  (3, 6727)	0.11258891678291157
  (3, 6739)	0.1807595567626873
  (3, 16245)	0.22689514706963246
  (3, 5732)	0.16534747561030483
  :	:
  (4930110, 12839)	0.24454476467972194
  (4930110, 16563)	0.13916

In [52]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8787051 stored elements and shape (1232529, 20000)>
  Coords	Values
  (0, 16893)	0.23517939836583793
  (0, 1179)	0.11743531994613883
  (0, 3208)	0.25776946235772663
  (0, 15289)	0.22478449714523566
  (0, 3705)	0.28539414984065087
  (0, 14232)	0.1891882953033847
  (0, 15372)	0.3459140286380915
  (0, 1273)	0.4167504670171682
  (0, 14247)	0.40547633453692655
  (0, 3253)	0.4888106726046092
  (1, 10149)	0.17866944494987655
  (1, 6233)	0.0886686554890349
  (1, 6627)	0.11196136762643558
  (1, 17341)	0.11433243784868861
  (1, 5652)	0.1278385636728456
  (1, 10379)	0.09658956777982702
  (1, 10094)	0.2918628767500205
  (1, 9205)	0.1124447458509327
  (1, 4801)	0.1547610091038671
  (1, 10823)	0.14574277671363492
  (1, 502)	0.14085927873399606
  (1, 4663)	0.17514630021748728
  (1, 4577)	0.30137909353437103
  (1, 7358)	0.39026236476983883
  (1, 3045)	0.1593011201337166
  :	:
  (1232523, 10149)	1.0
  (1232524, 8119)	0.1319103650551124
  (12

In [53]:
print(df.shape)
print(df['review_rating'].shape)

print(df_balanced['sentiment'].shape)


(2887975, 11)
(2887975,)
(6162645,)


In [54]:
#training the model
model = LogisticRegression(max_iter=1000)

In [55]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [56]:
#accuracy score on training model
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,y_train)

In [57]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.6116466630805442


In [58]:
#accuracy score on test model
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,y_test)

In [59]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.6073682647629386


In [60]:
baseline_accuracy = df_balanced['sentiment'].value_counts(normalize=True).max()
print(baseline_accuracy)


0.3333333333333333


In [61]:
import pickle

In [75]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [76]:
#loading saved model
loaded_model=pickle.load(open('trained_model.sav','rb'))


In [77]:
X_new=X_test[3]
print(y_test[3])
prediction = model.predict(X_new)
sentiment = le.inverse_transform(prediction)[0]
print("Predicted Sentiment:", sentiment)



2
Predicted Sentiment: positive review


In [78]:
X_new=X_test[11]
print(y_test[11])
prediction = model.predict(X_new)
sentiment = le.inverse_transform(prediction)[0]
print("Predicted Sentiment:", sentiment)



0
Predicted Sentiment: negative review


In [79]:
proba = model.predict_proba(X_new)
print("Class Probabilities:", proba)
print("Predicted Label:", le.inverse_transform([np.argmax(proba)]))





Class Probabilities: [[0.53129437 0.33311605 0.13558959]]
Predicted Label: ['negative review']
